Upload ExoGeni Credentials from ~/.ssl/user.jks


## Create ExoGeni Credentials 

In [ ]:
import base64
with open("user.jks", "rb") as image_file:
    exoGENI_keystore_as_base64 = base64.b64encode(image_file.read()).decode("utf-8") 


In [ ]:
import getpass
exoGENI_credential_user  = getpass.getpass('Enter your ExoGENI_credential_user')

In [ ]:
exoGENI_credential_token = getpass.getpass("Enter your ExoGENI_credential_token " )

## Create EC2 Credentials 

In [ ]:
aws_access_key_id  = getpass.getpass('Enter your aws_access_key_id')

In [ ]:
aws_secret_access_key  = getpass.getpass('Enter your aws_secret_access_key')

## Enter API URL

In [ ]:
base_url = input("Enter host base_url " )
# http://manager:8080
# http://localhost:30000
print(base_url)

## Upload Credentials

In [ ]:
import requests
path = "/credential"
payload = '{"cloud_provider_name": "ExoGENI","keys": {"keystore": "'+exoGENI_keystore_as_base64+'"}, "token": "'+exoGENI_credential_token+'",  "token_type": "password",  "user": "'+exoGENI_credential_user+'"}'
headers = {
  'Content-Type': 'application/json',
   'accept': 'application/json'
}
response = requests.post(base_url+path, data=payload,headers=headers)
credential_id = response.text
print(credential_id)

In [ ]:
path = "/credential"
payload = '{ "cloud_provider_name": "EC2","keys": {"aws_access_key_id": "'+aws_access_key_id+'"},  "token_type": "access_key", "token": "'+aws_secret_access_key+'"}' 
headers = {
  'Content-Type': 'application/json',
   'accept': 'application/json'
}
response = requests.post(base_url+path, data=payload,headers=headers)
credential_id = response.text
print(credential_id)

## Upload TOSCA

In [ ]:
path = "/tosca_template"
payload = {}
filename = 'lifeWatch_vre1.yaml'
fin = open(filename, 'rb')
files = {'file': fin}

headers = {}

response = requests.request("POST", base_url+path, headers=headers, files = files)
tosca_id = response.text
    

## Get TOSCA

In [ ]:
import yaml

path = "/tosca_template/"+tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload)
tosca = response.text
print(tosca)

## Show Topology

In [ ]:
import networkx as nx
import matplotlib
import plotly.graph_objects as go


def build_graph(node_templates):
    graph = nx.DiGraph()
    for node_name in node_templates:
        graph.add_node(node_name, attr_dict=node_templates[node_name])
        if 'requirements' in node_templates[node_name]:
            for req in node_templates[node_name]['requirements']:
                req_name = next(iter(req))
                req_node_name = req[req_name]['node']
                if 'relationship' in req[req_name] and 'type' in req[req_name]['relationship']:
                    relationship_type = req[req_name]['relationship']['type']
                else:
                    if 'relationship' not in req[req_name]:
                        relationship_type = 'tosca.relationships.DependsOn'
                    else:
                        relationship_type = req[req_name]['relationship']
                graph.add_edge(node_name, req_node_name, relationship=relationship_type)

    # nx.draw(graph, with_labels=True)
    # plt.savefig("/tmp/graph.png")
    # plt.show()
    return graph

def get_tosca(tosca_id):
    path = "/tosca_template/"+tosca_id
    payload = {}
    headers= {'accept':'text/plain'}
    response = requests.request("GET", base_url+path, headers=headers, data = payload)
    return response.text

tosca = get_tosca(tosca_id)
tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


## Make Plan

In [ ]:
path = "/planner/plan/"+tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload)
planed_tosca_id = response.text

In [ ]:
tosca = get_tosca(planed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if 'properties' in tosca_dict['topology_template']['node_templates'][node_name]:
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['properties']))
        print('---------------------------------------------------------------------------------------------------------------------')
        
        

## Provision Plan

In [ ]:
path = "/provisioner/provision/"+planed_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload)
provisioned_tosca_id = response.text

In [ ]:
tosca = get_tosca(provisioned_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)


for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.VM.Compute':
        print(node_name+': '+tosca_dict['topology_template']['node_templates'][node_name]['attributes']['public_ip'])

## Deploy K8s And WS

In [ ]:
path = "/deployer/deploy/"+provisioned_tosca_id
payload = {}
headers= {'accept':'text/plain'}

response = requests.request("GET", base_url+path, headers=headers, data = payload)
deployed_tosca_id = response.text

In [ ]:
tosca = get_tosca(deployed_tosca_id)

tosca_dict = yaml.safe_load(tosca)
graph = build_graph(tosca_dict['topology_template']['node_templates'])
nx.draw(graph, with_labels=True)

for node_name in tosca_dict['topology_template']['node_templates']:
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.docker.Orchestrator.Kubernetes':
        for token in tosca_dict['topology_template']['node_templates'][node_name]['attributes']['tokens']:
            if token['user'] == 'dashboard_token':
                print(node_name+': '+' dashboard_token: '+token['token'])
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['dashboard_url']))
    if tosca_dict['topology_template']['node_templates'][node_name]['type'] == 'tosca.nodes.QC.Container.Application.Docker':
        print(node_name+': '+str(tosca_dict['topology_template']['node_templates'][node_name]['attributes']['service_url']))
        